In [1]:
import sqlparse

In [2]:
the_query = '''select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') 
               Group by a1
               limit 15'''

In [3]:
res = sqlparse.parse(the_query)
stmt = res[0]
stmt.tokens

[<DML 'select' at 0x4E32BE8>,
 <Whitespace ' ' at 0x4E32DC8>,
 <IdentifierList 'a1, co...' at 0x4E494F8>,
 <Whitespace ' ' at 0x4D4A108>,
 <Newline ' ' at 0x4D4A168>,
 <Whitespace ' ' at 0x4D4A1C8>,
 <Whitespace ' ' at 0x4D4A228>,
 <Whitespace ' ' at 0x4D4A288>,
 <Whitespace ' ' at 0x4D4A2E8>,
 <Whitespace ' ' at 0x4D4A348>,
 <Whitespace ' ' at 0x4D4A3A8>,
 <Whitespace ' ' at 0x4D4A408>,
 <Whitespace ' ' at 0x4D4A468>,
 <Whitespace ' ' at 0x4D4A4C8>,
 <Whitespace ' ' at 0x4D4A528>,
 <Whitespace ' ' at 0x4D4A588>,
 <Whitespace ' ' at 0x4D4A5E8>,
 <Whitespace ' ' at 0x4D4A648>,
 <Whitespace ' ' at 0x4D4A6A8>,
 <Whitespace ' ' at 0x4D4A708>,
 <Keyword 'from' at 0x4D4A768>,
 <Whitespace ' ' at 0x4D4A7C8>,
 <Identifier 'MEOW.A...' at 0x4E490C0>,
 <Whitespace ' ' at 0x4D4AAC8>,
 <Newline ' ' at 0x4D4AB28>,
 <Whitespace ' ' at 0x4D4AB88>,
 <Whitespace ' ' at 0x4D4ABE8>,
 <Whitespace ' ' at 0x4D4AC48>,
 <Whitespace ' ' at 0x4D4ACA8>,
 <Whitespace ' ' at 0x4D4AD08>,
 <Whitespace ' ' at 0x4D4AD6

In [123]:
print(stmt.tokens[6].get_parent_name())
print(stmt.tokens[6].get_real_name())
print(stmt.tokens[6].get_name())
print(stmt.tokens[8].tokens)
print(stmt.tokens[8].tokens[2].token_first())
print(stmt.tokens[8].tokens[2].get_name())
print(stmt.tokens[8].tokens[2].value)
# i = 0
# for ii in stmt.tokens[8].tokens:
    
#     print(i)
#     i = i+1
#     try:
#         if(ii.value == "where"):
#             continue
#         print(ii.value)
#         print(ii.get_real_name())
#     except Exception as e :
#         pass

MEOW
AAA_table
t3
[<Keyword 'where' at 0x21CCFD1C168>, <Whitespace ' ' at 0x21CCFD1C1C8>, <Comparison 't3.X1 ...' at 0x21CCFE9F148>, <Whitespace ' ' at 0x21CCFD1C768>, <Keyword 'and' at 0x21CCFD1CDC8>, <Whitespace ' ' at 0x21CCFD1C708>, <Identifier 't3.X2' at 0x21CCFE9FAC8>, <Whitespace ' ' at 0x21CCF9D8948>, <Keyword 'in' at 0x21CCF9D8708>, <Whitespace ' ' at 0x21CCF9D8A68>, <Parenthesis '('a1',...' at 0x21CCFF57A48>, <Whitespace ' ' at 0x21CCFB576A8>]
t3.X1
X1
t3.X1 = 'thing'


In [89]:
tmp = stmt.tokens[8].tokens[2]

In [115]:
tmp.get_alias()

'X1'

# TABLE NAME

In [10]:
tables = []
parsed = sqlparse.parse(the_query)
stmt = parsed[0]
from_seen = False

In [4]:
## 資料表名稱萃取
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML


def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False


def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        print("=======item_FIRST===================")
        print(item.ttype)
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    print("x:===================")
                    print(x.ttype)
                    yield x
            elif item.ttype is Keyword:
                return
            else:
                print("item:===================")
                print(item.ttype)
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True


def extract_table_identifiers(token_stream):
    for item in token_stream:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                yield identifier.get_name()
        elif isinstance(item, Identifier):
            yield item.get_name()
        # It's a bug to check for Keyword here, but in the example
        # above some tables names are identified as keywords...
        elif item.ttype is Keyword:
            yield item.value


def extract_tables(sql):
    stream = extract_from_part(sqlparse.parse(sql)[0])
    return list(extract_table_identifiers(stream))


if __name__ == '__main__':
    sql = """
    select K.a,K.b from (select H.b from (select G.c from (select F.d from
    (select E.e from MEOW.A as t1 , MEOW.B, MEOW.C, MEOW.D, MEOW.E), F), G), H), I, J, K order by 1,2;
    """

    tables = ', '.join(extract_tables(sql))
    print('Tables: {0}'.format(tables))

=======item_FIRST===================
Token.Text.Whitespace.Newline
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Keyword.DML
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
None
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Keyword
=======item_FIRST===================
Token.Text.Whitespace
item:===================
Token.Text.Whitespace
=======item_FIRST===================
None
=======item_FIRST===================
Token.Punctuation
=======item_FIRST===================
Token.Keyword.DML
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
None
=======item_FIRST===================
Token.Text.Whitespace
=======item_F

In [ ]:
print(stmt.tokens[6].get_parent_name())
print(stmt.tokens[6].get_real_name())
print(stmt.tokens[6].get_name())

## 非槽狀FINAL

In [ ]:
def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        print("=======item_FIRST===================")
        print(item.ttype)
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    print("x:===================")
                    print(x.ttype)
                    yield x
            elif item.ttype is Keyword:
                return
            else:
                print("item:===================")
                print(item.ttype)
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True

In [161]:
query_tmp = "select t1.e from MEOW.A as t1 , MEOW.B as t2, MEOW.C as t3, MEOW.D as t4, QUERY.E as t5"

# print(', '.join(extract_tables(query_tmp)))
item_part = extract_from_part(sqlparse.parse(query_tmp)[0])

for item in item_part :
    print(item.ttype)
    if isinstance(item, IdentifierList):
        for identifier in item.get_identifiers():
            print(identifier.get_name() + "|"+ identifier.get_parent_name()+ "|"+ identifier.get_real_name())
#             print("===========================")
#             print(identifier.get_name() )
#             print(identifier.get_parent_name())
#             print(identifier.get_real_name())
    elif isinstance(item, Identifier):
        print(item.get_name())
    # It's a bug to check for Keyword here, but in the example
    # above some tables names are identified as keywords...
    elif item.ttype is Keyword:
        print(item.value)

=======item_FIRST===================
Token.Keyword.DML
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
None
=======item_FIRST===================
Token.Text.Whitespace
=======item_FIRST===================
Token.Keyword
=======item_FIRST===================
Token.Text.Whitespace
item:===================
Token.Text.Whitespace
Token.Text.Whitespace
=======item_FIRST===================
None
item:===================
None
None
t1|MEOW|A
t2|MEOW|B
t3|MEOW|C
t4|MEOW|D
t5|QUERY|E


# WHERE 
https://stackoverflow.com/questions/55398189/parse-sql-select-statement-to-fetch-the-where-clause-conditions-in-python

In [124]:
the_query = "select a1, a2, a3 from MEOW.AAA_table as t3 where t3.X1 = 'thing' and t3.X2 in ('a1', 'a2', 'a3') limit 15"
res = sqlparse.parse(the_query)
stmt = res[0]
stmt.tokens
print(stmt.tokens[6].get_parent_name())
print(stmt.tokens[6].get_real_name())
print(stmt.tokens[6].get_name())
print(stmt.tokens[8].tokens)
print(stmt.tokens[8].tokens[2].token_first())
print(stmt.tokens[8].tokens[2].get_name())
print(stmt.tokens[8].tokens[2].value)

MEOW
AAA_table
t3
[<Keyword 'where' at 0x21CD7150CA8>, <Whitespace ' ' at 0x21CD71DA048>, <Comparison 't3.X1 ...' at 0x21CD72096C8>, <Whitespace ' ' at 0x21CD7210408>, <Keyword 'and' at 0x21CD71759A8>, <Whitespace ' ' at 0x21CD7175588>, <Identifier 't3.X2' at 0x21CD7209248>, <Whitespace ' ' at 0x21CD7175468>, <Keyword 'in' at 0x21CD71755E8>, <Whitespace ' ' at 0x21CD7175828>, <Parenthesis '('a1',...' at 0x21CD72091C8>, <Whitespace ' ' at 0x21CD7218528>]
t3.X1
X1
t3.X1 = 'thing'


In [151]:
import sqlparse
import re
s = "select count(*) from users as t1 where t1.employee_type = 'Employee' AND (t1.employment_status = 'Active' OR t1.employment_status = 'On Leave') AND (t1.time_type='Full time' OR t1.country_code <> 'US') AND t1.hire_date < NOW() AND t1.email_work IS NOT NULL AND LENGTH(t1.email_work) > 0 AND NOT (t1.ob_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE') AND t1.job_profile_id NOT IN ('20992', '20993', '20994', '20995', '20996', '20997') AND t1.country_code NOT IN ('CN', 'MO', 'SG', 'MY', 'TH', 'VN', 'MM', 'KH', 'PH', 'ID')"

parsed = sqlparse.parse(s)
where = parsed[0][-1]

sql_tokens = []
def get_tokens(where):
    for i in where.tokens:
        try:
            ### IS NOT 系列處理
            
            ### () 系列處理
            
            token_first_list = re.split("\\.",str(i.token_first()))
            if(len(token_first_list)==1):
                parentname = "NO_PARENT"
            else:
                parentname = token_first_list[0]
            name = i.get_real_name()
            if name and not isinstance(i, sqlparse.sql.Parenthesis):
                # sql_tokens.append("{0} - {1} - {2}".format(str(i), str(name), i.value))
                sql_tokens.append({
                    'token_first':str(i.token_first()),
                    'parent':str(parentname),
                    'key': str(name),
                    'value': i.value,
                })
            else:
                get_tokens(i)
        except Exception as e:
            pass


get_tokens(where)
for i in sql_tokens:
    print (i)

{'token_first': 't1.employee_type', 'parent': 't1', 'key': 'employee_type', 'value': "t1.employee_type = 'Employee'"}
{'token_first': 't1.employment_status', 'parent': 't1', 'key': 'employment_status', 'value': "t1.employment_status = 'Active'"}
{'token_first': 't1.employment_status', 'parent': 't1', 'key': 'employment_status', 'value': "t1.employment_status = 'On Leave'"}
{'token_first': 't1.time_type', 'parent': 't1', 'key': 'time_type', 'value': "t1.time_type='Full time'"}
{'token_first': 't1.country_code', 'parent': 't1', 'key': 'country_code', 'value': "t1.country_code <> 'US'"}
{'token_first': 't1.hire_date', 'parent': 't1', 'key': 'hire_date', 'value': 't1.hire_date < NOW()'}
{'token_first': 't1', 'parent': 'NO_PARENT', 'key': 'email_work', 'value': 't1.email_work'}
{'token_first': 'LENGTH(t1.email_work)', 'parent': 'LENGTH(t1', 'key': 'LENGTH', 'value': 'LENGTH(t1.email_work) > 0'}
{'token_first': 't1', 'parent': 'NO_PARENT', 'key': 'ob_profile_id', 'value': 't1.ob_profile_id'}

In [140]:
str(where.tokens[2].token_first())

't1.employee_type'

In [149]:
for tt in where.tokens:
    print (tt.value)

where
 
t1.employee_type = 'Employee'
 
AND
 
(t1.employment_status = 'Active' OR t1.employment_status = 'On Leave')
 
AND
 
(t1.time_type='Full time' OR t1.country_code <> 'US')
 
AND
 
t1.hire_date < NOW()
 
AND
 
t1.email_work
 
IS
 
NOT NULL
 
AND
 
LENGTH(t1.email_work) > 0
 
AND
 
NOT
 
(t1.ob_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE')
 
AND
 
t1.job_profile_id
 
NOT
 
IN
 
('20992', '20993', '20994', '20995', '20996', '20997')
 
AND
 
t1.country_code
 
NOT
 
IN
 
('CN', 'MO', 'SG', 'MY', 'TH', 'VN', 'MM', 'KH', 'PH', 'ID')


In [153]:
where.value

"where t1.employee_type = 'Employee' AND (t1.employment_status = 'Active' OR t1.employment_status = 'On Leave') AND (t1.time_type='Full time' OR t1.country_code <> 'US') AND t1.hire_date < NOW() AND t1.email_work IS NOT NULL AND LENGTH(t1.email_work) > 0 AND NOT (t1.ob_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE') AND t1.job_profile_id NOT IN ('20992', '20993', '20994', '20995', '20996', '20997') AND t1.country_code NOT IN ('CN', 'MO', 'SG', 'MY', 'TH', 'VN', 'MM', 'KH', 'PH', 'ID')"

In [152]:
where.tokens

[<Keyword 'where' at 0x21CD6F79BE8>,
 <Whitespace ' ' at 0x21CD6F79AC8>,
 <Comparison 't1.emp...' at 0x21CD6F9A148>,
 <Whitespace ' ' at 0x21CD6F79A68>,
 <Keyword 'AND' at 0x21CD6F79A08>,
 <Whitespace ' ' at 0x21CD6F799A8>,
 <Parenthesis '(t1.em...' at 0x21CD721D6C8>,
 <Whitespace ' ' at 0x21CD6FFC5E8>,
 <Keyword 'AND' at 0x21CD6FFC4C8>,
 <Whitespace ' ' at 0x21CD6FFC0A8>,
 <Parenthesis '(t1.ti...' at 0x21CD721D348>,
 <Whitespace ' ' at 0x21CCFB688E8>,
 <Keyword 'AND' at 0x21CCFB68CA8>,
 <Whitespace ' ' at 0x21CCFB68B28>,
 <Comparison 't1.hir...' at 0x21CD6F9A648>,
 <Whitespace ' ' at 0x21CD7210048>,
 <Keyword 'AND' at 0x21CD7210B88>,
 <Whitespace ' ' at 0x21CD72106A8>,
 <Identifier 't1.ema...' at 0x21CD6F9AE48>,
 <Whitespace ' ' at 0x21CD7210E88>,
 <Keyword 'IS' at 0x21CD7210768>,
 <Whitespace ' ' at 0x21CD72109A8>,
 <Keyword 'NOT NU...' at 0x21CD7210648>,
 <Whitespace ' ' at 0x21CD7210EE8>,
 <Keyword 'AND' at 0x21CD70D4468>,
 <Whitespace ' ' at 0x21CD70D41C8>,
 <Comparison 'LENGTH...

## 改寫變得更結構化

In [ ]:
import sqlparse
import re
s = "select count(*) from users as t1 where t1.employee_type = 'Employee' AND (t1.employment_status = 'Active' OR t1.employment_status = 'On Leave') AND (t1.time_type='Full time' OR t1.country_code <> 'US') AND t1.hire_date < NOW() AND t1.email_work IS NOT NULL AND LENGTH(t1.email_work) > 0 AND NOT (t1.ob_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE') AND t1.job_profile_id NOT IN ('20992', '20993', '20994', '20995', '20996', '20997') AND t1.country_code NOT IN ('CN', 'MO', 'SG', 'MY', 'TH', 'VN', 'MM', 'KH', 'PH', 'ID')"

parsed = sqlparse.parse(s)
where = parsed[0][-1]

sql_tokens = []
def get_tokens(where):
    for i in where.tokens:
        try:
            token_first_list = re.split("\\.",str(i.token_first()))
            if(len(token_first_list)==1):
                parentname = "NO_PARENT"
            else:
                parentname = token_first_list[0]
            name = i.get_real_name()
            if name and not isinstance(i, sqlparse.sql.Parenthesis):
                # sql_tokens.append("{0} - {1} - {2}".format(str(i), str(name), i.value))
                sql_tokens.append({
                    'parent':str(parentname),
                    'key': str(name),
                    'value': i.value,
                })
            else:
                get_tokens(i)
        except Exception as e:
            pass


get_tokens(where)
for i in sql_tokens:
    print (i)

## 加入 LIST 系列

In [133]:
import sqlparse

s = "select count(*) from users where employee_type = 'Employee' AND (employment_status = 'Active' OR employment_status = 'On Leave') AND (time_type='Full time' OR country_code <> 'US') AND hire_date < NOW() AND email_work IS NOT NULL AND LENGTH(email_work) > 0 AND NOT (job_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE') AND job_profile_id NOT IN ('20992', '20993', '20994', '20995', '20996', '20997') AND country_code NOT IN ('CN', 'MO', 'SG', 'MY', 'TH', 'VN', 'MM', 'KH', 'PH', 'ID')"

parsed = sqlparse.parse(s)
where = parsed[0][-1]

sql_tokens = []
def get_tokens(where):
    identifier = None
    for i in where.tokens:
        try:
            name = i.get_real_name()
            if name and isinstance(i, sqlparse.sql.Identifier):
                identifier = i
            elif identifier and isinstance(i, sqlparse.sql.Parenthesis):
                sql_tokens.append({
                    'key': str(identifier),
                    'value': token.value
                })
            elif name:
                identifier = None
                # sql_tokens.append("{0} - {1} - {2}".format(str(i), str(name), i.value))
                sql_tokens.append({
                    'key': str(name),
                    'value': u''.join(token.value for token in i.flatten()),
                })
            else:
                get_tokens(i)
        except Exception as e:
            pass

get_tokens(where)
for i in sql_tokens:
    print (i)

{'key': 'employee_type', 'value': "employee_type = 'Employee'"}
{'key': 'employment_status', 'value': "employment_status = 'Active'"}
{'key': 'employment_status', 'value': "employment_status = 'On Leave'"}
{'key': 'time_type', 'value': "time_type='Full time'"}
{'key': 'country_code', 'value': "country_code <> 'US'"}
{'key': 'hire_date', 'value': 'hire_date < NOW()'}
{'key': 'LENGTH', 'value': 'LENGTH(email_work) > 0'}
{'key': 'job_profile_id', 'value': "(job_profile_id IN ('8802 - Comm Ops - 1', '8801 - CityOps - 2', '10034', '10455', '21014', '21015', '21016', '21018', '21017', '21019') AND country_code = 'IE')"}
{'key': 'job_profile_id', 'value': 'where'}
{'key': 'country_code', 'value': 'where'}


In [73]:
where

<Where 'where ...' at 0x21CCF736848>

In [112]:
import re 
re.split("\\.","xxx")

['xxx']